# RLHF 與 LLM 對齊 (RLHF and LLM Alignment)

本 notebook 對應李宏毅老師 2025 Spring ML HW7，深入探討如何使用人類反饋強化學習（RLHF）來對齊大型語言模型。

## 學習目標

1. 理解 LLM 對齊問題與 RLHF 的動機
2. 掌握 Reward Model 的訓練方法
3. 了解 PPO 在 LLM 上的應用
4. 學習 DPO（Direct Preference Optimization）
5. 使用 TRL 庫進行實作

## 參考資源

- [InstructGPT Paper](https://arxiv.org/abs/2203.02155) - OpenAI RLHF 原始論文
- [DPO Paper](https://arxiv.org/abs/2305.18290) - Direct Preference Optimization
- [TRL Documentation](https://huggingface.co/docs/trl) - Transformer Reinforcement Learning
- [2025 Spring HW7](https://speech.ee.ntu.edu.tw/~hylee/ml/2025-spring.php)

## 1. 為什麼需要對齊？(Why Alignment?)

### 1.1 預訓練 LLM 的問題

預訓練的 LLM 透過預測下一個 token 學習語言，但這不等於：

```
┌─────────────────────────────────────────────────────────────────┐
│                    預訓練 vs 使用者期望                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  預訓練目標：P(next_token | context)                             │
│  ↓                                                              │
│  模型學到：                                                      │
│  - 統計模式（網路文本的分布）                                      │
│  - 各種風格（包含有害內容）                                        │
│  - 事實與虛構混合                                                 │
│                                                                 │
│  使用者期望：                                                     │
│  - 有幫助 (Helpful)                                             │
│  - 無害 (Harmless)                                              │
│  - 誠實 (Honest)                                                │
│                                                                 │
│  → 存在 "對齊差距" (Alignment Gap)                               │
└─────────────────────────────────────────────────────────────────┘
```

### 1.2 對齊的目標

**HHH 原則**（Anthropic 提出）：
- **Helpful（有幫助）**：回答問題、完成任務
- **Harmless（無害）**：不產生有害、歧視、危險內容
- **Honest（誠實）**：承認不確定性、不編造事實

## 2. RLHF 流程概覽

### 2.1 三階段訓練流程

```
┌─────────────────────────────────────────────────────────────────────────┐
│                      RLHF 三階段流程 (InstructGPT)                       │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Stage 1: Supervised Fine-Tuning (SFT)                                 │
│  ┌─────────────────────────────────────────────────────────────┐       │
│  │  Human Demonstrations                                        │       │
│  │  (prompt, ideal_response) pairs                             │       │
│  │           ↓                                                  │       │
│  │  Pretrained LLM  ──────→  SFT Model                         │       │
│  │  (GPT-3)                  (follows instructions)            │       │
│  └─────────────────────────────────────────────────────────────┘       │
│                              │                                          │
│                              ▼                                          │
│  Stage 2: Reward Model Training                                        │
│  ┌─────────────────────────────────────────────────────────────┐       │
│  │  SFT Model generates multiple responses per prompt          │       │
│  │           ↓                                                  │       │
│  │  Human ranks responses: A > B > C > D                       │       │
│  │           ↓                                                  │       │
│  │  Train Reward Model: r(prompt, response) → scalar           │       │
│  └─────────────────────────────────────────────────────────────┘       │
│                              │                                          │
│                              ▼                                          │
│  Stage 3: RL Fine-Tuning (PPO)                                         │
│  ┌─────────────────────────────────────────────────────────────┐       │
│  │  Environment: Prompts from dataset                          │       │
│  │  Policy: SFT Model (being optimized)                        │       │
│  │  Reward: Reward Model output                                │       │
│  │           ↓                                                  │       │
│  │  Optimize: max E[r(x,y)] - β·KL(π||π_ref)                   │       │
│  │           ↓                                                  │       │
│  │  Final RLHF Model                                           │       │
│  └─────────────────────────────────────────────────────────────┘       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
import random

# 設定隨機種子
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 3. Reward Model 訓練

### 3.1 偏好資料格式

Reward Model 學習人類偏好，訓練資料格式：

```
┌─────────────────────────────────────────────────────────────┐
│                   偏好資料 (Preference Data)                 │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Prompt: "What is the capital of France?"                  │
│                                                             │
│  Response A (Chosen/Winner):                               │
│  "The capital of France is Paris. It is located in        │
│   northern France and is known for..."                     │
│                                                             │
│  Response B (Rejected/Loser):                              │
│  "France capital Paris maybe."                             │
│                                                             │
│  Human Preference: A > B                                   │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 3.2 Bradley-Terry Model

Reward Model 使用 Bradley-Terry 模型來建模偏好：

$$P(y_w \succ y_l | x) = \sigma(r(x, y_w) - r(x, y_l))$$

其中 $\sigma$ 是 sigmoid 函數，$r(x, y)$ 是 reward model 對 (prompt, response) 的評分。

In [ ]:
@dataclass
class PreferenceExample:
    """偏好資料範例"""
    prompt: str
    chosen: str      # 人類偏好的回應
    rejected: str    # 被拒絕的回應


# 範例偏好資料
preference_data = [
    PreferenceExample(
        prompt="Explain what machine learning is.",
        chosen="Machine learning is a subset of artificial intelligence that enables computers to learn patterns from data without being explicitly programmed. It involves algorithms that improve through experience.",
        rejected="ML is computers doing stuff."
    ),
    PreferenceExample(
        prompt="How do I make coffee?",
        chosen="To make coffee: 1) Boil water, 2) Add 2 tablespoons of ground coffee per cup to your filter, 3) Pour hot water over the grounds, 4) Let it brew for 3-4 minutes, 5) Enjoy!",
        rejected="Just put coffee in water."
    ),
    PreferenceExample(
        prompt="Is the Earth flat?",
        chosen="No, the Earth is not flat. Scientific evidence from multiple sources including satellite imagery, physics, and direct observation confirms that Earth is an oblate spheroid (slightly flattened sphere).",
        rejected="Some people say yes, some say no. Who knows?"
    ),
]

print("偏好資料範例：")
for i, ex in enumerate(preference_data):
    print(f"\n--- Example {i+1} ---")
    print(f"Prompt: {ex.prompt}")
    print(f"Chosen: {ex.chosen[:50]}...")
    print(f"Rejected: {ex.rejected[:50]}...")

In [ ]:
class SimpleRewardModel(nn.Module):
    """
    簡化版 Reward Model
    
    實際的 Reward Model 是在 LLM 基礎上加一個 scalar head，
    這裡使用簡化版本來展示概念。
    """
    def __init__(self, vocab_size: int, embed_dim: int, hidden_dim: int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # 簡單的 Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=4,
            dim_feedforward=hidden_dim,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)
        
        # Reward head: 將序列表示映射到 scalar
        self.reward_head = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor = None) -> torch.Tensor:
        """
        Args:
            input_ids: [batch_size, seq_len]
            attention_mask: [batch_size, seq_len]
        Returns:
            rewards: [batch_size, 1]
        """
        # Embedding
        x = self.embedding(input_ids)  # [batch, seq, embed]
        
        # Transformer encoding
        if attention_mask is not None:
            # Convert to transformer mask format
            src_key_padding_mask = ~attention_mask.bool()
        else:
            src_key_padding_mask = None
        
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)
        
        # 取最後一個 token 的表示（或平均）
        if attention_mask is not None:
            # Masked mean pooling
            mask_expanded = attention_mask.unsqueeze(-1).float()
            sum_embeddings = (x * mask_expanded).sum(dim=1)
            pooled = sum_embeddings / mask_expanded.sum(dim=1).clamp(min=1e-9)
        else:
            pooled = x.mean(dim=1)
        
        # 計算 reward
        reward = self.reward_head(pooled)
        return reward


# 測試 Reward Model
vocab_size = 1000
embed_dim = 128
hidden_dim = 256

reward_model = SimpleRewardModel(vocab_size, embed_dim, hidden_dim).to(device)

# 模擬輸入
batch_size = 4
seq_len = 32
dummy_input = torch.randint(0, vocab_size, (batch_size, seq_len)).to(device)
dummy_mask = torch.ones(batch_size, seq_len).to(device)

rewards = reward_model(dummy_input, dummy_mask)
print(f"Reward shape: {rewards.shape}")
print(f"Sample rewards: {rewards.squeeze().tolist()}")

In [ ]:
def reward_model_loss(reward_chosen: torch.Tensor, 
                      reward_rejected: torch.Tensor) -> torch.Tensor:
    """
    Reward Model 的損失函數（Bradley-Terry loss）
    
    L = -log(σ(r_chosen - r_rejected))
    
    希望 chosen response 的 reward 高於 rejected response
    """
    # 計算 reward 差異
    reward_diff = reward_chosen - reward_rejected
    
    # Binary cross entropy: -log(sigmoid(diff))
    loss = -F.logsigmoid(reward_diff).mean()
    
    return loss


def compute_reward_accuracy(reward_chosen: torch.Tensor, 
                            reward_rejected: torch.Tensor) -> float:
    """
    計算 Reward Model 的準確率
    正確 = r(chosen) > r(rejected)
    """
    correct = (reward_chosen > reward_rejected).float().mean()
    return correct.item()


# 示範損失計算
r_chosen = torch.tensor([[2.5], [1.8], [3.2]])
r_rejected = torch.tensor([[1.2], [2.1], [0.8]])

loss = reward_model_loss(r_chosen, r_rejected)
acc = compute_reward_accuracy(r_chosen, r_rejected)

print(f"Reward differences: {(r_chosen - r_rejected).squeeze().tolist()}")
print(f"Loss: {loss.item():.4f}")
print(f"Accuracy: {acc:.2%}")

## 4. PPO 用於 LLM 對齊

### 4.1 PPO 回顧

PPO (Proximal Policy Optimization) 是一種 policy gradient 方法，在 RLHF 中：

```
┌─────────────────────────────────────────────────────────────────────┐
│                    PPO for LLM 對齊                                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  State (s):    Prompt x + generated tokens so far                  │
│  Action (a):   Next token to generate                              │
│  Policy π(a|s): LLM probability distribution over vocabulary        │
│  Reward (r):   Reward Model output (only at end of response)       │
│                                                                     │
│  ┌─────────────────────────────────────────────────────────────┐   │
│  │                                                             │   │
│  │  Prompt    Policy (LLM)    Response    Reward Model         │   │
│  │    x    →     π_θ      →     y     →      r(x,y)           │   │
│  │                                              ↓              │   │
│  │                                          Scalar             │   │
│  │                                          Reward             │   │
│  │                                                             │   │
│  └─────────────────────────────────────────────────────────────┘   │
│                                                                     │
│  PPO Objective:                                                    │
│  L_PPO = E[min(r_t(θ)·Â_t, clip(r_t(θ), 1-ε, 1+ε)·Â_t)]          │
│                                                                     │
│  where r_t(θ) = π_θ(a_t|s_t) / π_θ_old(a_t|s_t)                   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 4.2 KL Penalty

為了防止模型偏離太遠，加入 KL divergence 懲罰：

$$\text{Objective} = \mathbb{E}[r(x, y)] - \beta \cdot KL(\pi_\theta \| \pi_{ref})$$

其中 $\pi_{ref}$ 是原始 SFT model。

In [ ]:
class SimplePPOTrainer:
    """
    簡化版 PPO Trainer for LLM
    展示核心概念，實際使用請參考 TRL 庫
    """
    def __init__(
        self,
        policy_model: nn.Module,
        ref_model: nn.Module,
        reward_model: nn.Module,
        kl_coef: float = 0.1,
        clip_range: float = 0.2,
        value_clip_range: float = 0.2,
    ):
        self.policy = policy_model
        self.ref_model = ref_model
        self.reward_model = reward_model
        self.kl_coef = kl_coef
        self.clip_range = clip_range
        self.value_clip_range = value_clip_range
        
        # Freeze reference model
        for param in self.ref_model.parameters():
            param.requires_grad = False
        for param in self.reward_model.parameters():
            param.requires_grad = False
    
    def compute_kl_divergence(self, policy_logprobs: torch.Tensor, 
                               ref_logprobs: torch.Tensor) -> torch.Tensor:
        """
        計算 KL divergence: KL(π_θ || π_ref)
        使用 log probabilities 的近似
        """
        kl = policy_logprobs - ref_logprobs
        return kl.mean()
    
    def compute_advantages(self, rewards: torch.Tensor, 
                          values: torch.Tensor,
                          gamma: float = 1.0,
                          lam: float = 0.95) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        計算 GAE (Generalized Advantage Estimation)
        
        對於 LLM，通常只在 response 結束時給一個 reward，
        所以這裡簡化處理
        """
        # 簡化版：advantage = reward - value (baseline)
        advantages = rewards - values
        returns = rewards
        return advantages, returns
    
    def ppo_loss(self, 
                 old_logprobs: torch.Tensor,
                 new_logprobs: torch.Tensor,
                 advantages: torch.Tensor) -> torch.Tensor:
        """
        PPO Clipped Objective
        """
        # Probability ratio
        ratio = torch.exp(new_logprobs - old_logprobs)
        
        # Clipped ratio
        clipped_ratio = torch.clamp(ratio, 
                                     1 - self.clip_range, 
                                     1 + self.clip_range)
        
        # PPO loss: negative because we want to maximize
        loss1 = ratio * advantages
        loss2 = clipped_ratio * advantages
        loss = -torch.min(loss1, loss2).mean()
        
        return loss


print("PPO Trainer 類別已定義")
print("\n核心組件：")
print("1. Policy Model - 要優化的 LLM")
print("2. Reference Model - 原始 SFT Model（凍結）")
print("3. Reward Model - 評估回應品質")
print("4. KL Penalty - 防止偏離太遠")

In [ ]:
# 視覺化 PPO Clipping
def visualize_ppo_clipping(clip_range=0.2):
    """視覺化 PPO clipping 機制"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Ratio values
    ratios = np.linspace(0.5, 1.5, 200)
    
    # Case 1: Positive advantage (want to increase probability)
    advantage_pos = 1.0
    
    unclipped_pos = ratios * advantage_pos
    clipped_ratios = np.clip(ratios, 1 - clip_range, 1 + clip_range)
    clipped_pos = clipped_ratios * advantage_pos
    ppo_objective_pos = np.minimum(unclipped_pos, clipped_pos)
    
    axes[0].plot(ratios, unclipped_pos, 'b--', label='Unclipped', alpha=0.7)
    axes[0].plot(ratios, clipped_pos, 'r--', label='Clipped', alpha=0.7)
    axes[0].plot(ratios, ppo_objective_pos, 'g-', label='PPO Objective', linewidth=2)
    axes[0].axvline(x=1, color='gray', linestyle=':', alpha=0.5)
    axes[0].axvline(x=1-clip_range, color='orange', linestyle='--', alpha=0.5, label=f'Clip bounds ({1-clip_range}, {1+clip_range})')
    axes[0].axvline(x=1+clip_range, color='orange', linestyle='--', alpha=0.5)
    axes[0].set_xlabel('Probability Ratio (π_new / π_old)')
    axes[0].set_ylabel('Objective')
    axes[0].set_title('Positive Advantage (A > 0)\n"Good action, want higher probability"')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Case 2: Negative advantage (want to decrease probability)
    advantage_neg = -1.0
    
    unclipped_neg = ratios * advantage_neg
    clipped_neg = clipped_ratios * advantage_neg
    ppo_objective_neg = np.minimum(unclipped_neg, clipped_neg)
    
    axes[1].plot(ratios, unclipped_neg, 'b--', label='Unclipped', alpha=0.7)
    axes[1].plot(ratios, clipped_neg, 'r--', label='Clipped', alpha=0.7)
    axes[1].plot(ratios, ppo_objective_neg, 'g-', label='PPO Objective', linewidth=2)
    axes[1].axvline(x=1, color='gray', linestyle=':', alpha=0.5)
    axes[1].axvline(x=1-clip_range, color='orange', linestyle='--', alpha=0.5)
    axes[1].axvline(x=1+clip_range, color='orange', linestyle='--', alpha=0.5)
    axes[1].set_xlabel('Probability Ratio (π_new / π_old)')
    axes[1].set_ylabel('Objective')
    axes[1].set_title('Negative Advantage (A < 0)\n"Bad action, want lower probability"')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nPPO Clipping 說明：")
    print("- 當 A > 0（好動作）：限制 ratio 增加的幅度，防止過度更新")
    print("- 當 A < 0（壞動作）：限制 ratio 減少的幅度，保持穩定")
    print("- 這確保每次更新都是 'proximal'（接近的）")

visualize_ppo_clipping()

## 5. DPO: Direct Preference Optimization

### 5.1 DPO 的動機

RLHF 的問題：
- 需要訓練獨立的 Reward Model
- PPO 訓練複雜且不穩定
- 需要同時運行多個模型（memory 消耗大）

DPO 的核心洞察：**可以直接從偏好資料優化 policy，不需要顯式的 reward model！**

### 5.2 DPO 推導

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         DPO 推導過程                                     │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Step 1: RLHF 最優解的形式                                               │
│  ─────────────────────────────                                          │
│  對於 RLHF objective: max E[r(x,y)] - β·KL(π||π_ref)                   │
│                                                                         │
│  最優 policy 有封閉解：                                                  │
│  π*(y|x) = (1/Z(x)) · π_ref(y|x) · exp(r(x,y)/β)                       │
│                                                                         │
│  Step 2: 從 policy 反推 reward                                          │
│  ─────────────────────────────                                          │
│  重排得：                                                                │
│  r(x,y) = β · log(π*(y|x) / π_ref(y|x)) + β·log(Z(x))                 │
│                                                                         │
│  Step 3: 代入 Bradley-Terry Model                                       │
│  ─────────────────────────────                                          │
│  P(y_w > y_l | x) = σ(r(x,y_w) - r(x,y_l))                             │
│                                                                         │
│  代入後（Z(x) 被消掉）：                                                  │
│  P(y_w > y_l | x) = σ(β·[log(π(y_w|x)/π_ref(y_w|x))                   │
│                         - log(π(y_l|x)/π_ref(y_l|x))])                 │
│                                                                         │
│  Step 4: DPO Loss                                                       │
│  ─────────────────                                                      │
│  L_DPO = -E[log σ(β·(log(π(y_w|x)/π_ref(y_w|x))                        │
│                   - log(π(y_l|x)/π_ref(y_l|x))))]                       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
def dpo_loss(
    policy_chosen_logps: torch.Tensor,
    policy_rejected_logps: torch.Tensor,
    reference_chosen_logps: torch.Tensor,
    reference_rejected_logps: torch.Tensor,
    beta: float = 0.1,
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    計算 DPO Loss
    
    Args:
        policy_chosen_logps: log π_θ(y_w|x)
        policy_rejected_logps: log π_θ(y_l|x)
        reference_chosen_logps: log π_ref(y_w|x)
        reference_rejected_logps: log π_ref(y_l|x)
        beta: temperature parameter
    
    Returns:
        loss, chosen_rewards, rejected_rewards
    """
    # 計算 log ratios
    chosen_logratios = policy_chosen_logps - reference_chosen_logps
    rejected_logratios = policy_rejected_logps - reference_rejected_logps
    
    # Implicit rewards (這就是 DPO 隱式學到的 reward)
    chosen_rewards = beta * chosen_logratios
    rejected_rewards = beta * rejected_logratios
    
    # DPO loss: -log(σ(β * (log_ratio_chosen - log_ratio_rejected)))
    logits = chosen_logratios - rejected_logratios
    loss = -F.logsigmoid(beta * logits).mean()
    
    return loss, chosen_rewards.detach(), rejected_rewards.detach()


# 示範 DPO loss 計算
batch_size = 4

# 模擬 log probabilities
policy_chosen = torch.randn(batch_size)      # log π_θ(y_w|x)
policy_rejected = torch.randn(batch_size)    # log π_θ(y_l|x)
ref_chosen = torch.randn(batch_size)         # log π_ref(y_w|x)
ref_rejected = torch.randn(batch_size)       # log π_ref(y_l|x)

loss, chosen_r, rejected_r = dpo_loss(
    policy_chosen, policy_rejected, 
    ref_chosen, ref_rejected,
    beta=0.1
)

print(f"DPO Loss: {loss.item():.4f}")
print(f"Chosen implicit rewards: {chosen_r.tolist()}")
print(f"Rejected implicit rewards: {rejected_r.tolist()}")
print(f"Reward margins (chosen - rejected): {(chosen_r - rejected_r).tolist()}")

In [ ]:
class DPOTrainer:
    """
    DPO Trainer for LLM
    簡化版實現，展示核心概念
    """
    def __init__(
        self,
        model: nn.Module,
        ref_model: nn.Module,
        beta: float = 0.1,
        learning_rate: float = 1e-6,
    ):
        self.model = model
        self.ref_model = ref_model
        self.beta = beta
        
        # Freeze reference model
        for param in self.ref_model.parameters():
            param.requires_grad = False
        
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(), 
            lr=learning_rate
        )
    
    def get_batch_logps(
        self, 
        model: nn.Module,
        input_ids: torch.Tensor,
        labels: torch.Tensor,
        attention_mask: torch.Tensor = None,
    ) -> torch.Tensor:
        """
        計算整個序列的 log probability
        """
        # 這是簡化版，實際需要根據模型架構調整
        logits = model(input_ids)  # [batch, seq, vocab]
        
        # Shift for language modeling
        shift_logits = logits[:, :-1, :].contiguous()
        shift_labels = labels[:, 1:].contiguous()
        
        # Per-token log probabilities
        log_probs = F.log_softmax(shift_logits, dim=-1)
        
        # Gather the log probs of the actual tokens
        token_log_probs = log_probs.gather(-1, shift_labels.unsqueeze(-1)).squeeze(-1)
        
        # Sum over sequence (or apply mask)
        if attention_mask is not None:
            mask = attention_mask[:, 1:]
            token_log_probs = token_log_probs * mask
            sequence_log_probs = token_log_probs.sum(dim=-1) / mask.sum(dim=-1).clamp(min=1)
        else:
            sequence_log_probs = token_log_probs.mean(dim=-1)
        
        return sequence_log_probs
    
    def train_step(
        self,
        chosen_ids: torch.Tensor,
        rejected_ids: torch.Tensor,
        chosen_mask: torch.Tensor = None,
        rejected_mask: torch.Tensor = None,
    ) -> Dict[str, float]:
        """
        執行一步 DPO 訓練
        """
        self.optimizer.zero_grad()
        
        # 計算 policy model 的 log probs
        policy_chosen_logps = self.get_batch_logps(
            self.model, chosen_ids, chosen_ids, chosen_mask
        )
        policy_rejected_logps = self.get_batch_logps(
            self.model, rejected_ids, rejected_ids, rejected_mask
        )
        
        # 計算 reference model 的 log probs (no grad)
        with torch.no_grad():
            ref_chosen_logps = self.get_batch_logps(
                self.ref_model, chosen_ids, chosen_ids, chosen_mask
            )
            ref_rejected_logps = self.get_batch_logps(
                self.ref_model, rejected_ids, rejected_ids, rejected_mask
            )
        
        # 計算 DPO loss
        loss, chosen_rewards, rejected_rewards = dpo_loss(
            policy_chosen_logps, policy_rejected_logps,
            ref_chosen_logps, ref_rejected_logps,
            beta=self.beta
        )
        
        # 反向傳播
        loss.backward()
        self.optimizer.step()
        
        # 計算指標
        reward_margin = (chosen_rewards - rejected_rewards).mean().item()
        accuracy = (chosen_rewards > rejected_rewards).float().mean().item()
        
        return {
            'loss': loss.item(),
            'reward_margin': reward_margin,
            'accuracy': accuracy,
            'chosen_reward': chosen_rewards.mean().item(),
            'rejected_reward': rejected_rewards.mean().item(),
        }


print("DPO Trainer 已定義")
print("\nDPO vs RLHF (PPO):")
print("┌─────────────────────────────────────────────────────────────┐")
print("│              RLHF (PPO)           │           DPO           │")
print("├─────────────────────────────────────────────────────────────┤")
print("│  需要 Reward Model               │  不需要 Reward Model     │")
print("│  需要 Value Network              │  不需要 Value Network    │")
print("│  PPO 訓練複雜                    │  簡單的 supervised loss │")
print("│  需要 online sampling            │  可以完全 offline       │")
print("│  4 個模型（policy, ref, RM, V）  │  2 個模型（policy, ref）│")
print("│  訓練不穩定                      │  穩定收斂               │")
print("└─────────────────────────────────────────────────────────────┘")

## 6. DPO 變體與進階方法

### 6.1 IPO (Identity Preference Optimization)

DPO 可能過度擬合偏好資料，IPO 使用不同的損失函數來改善。

### 6.2 KTO (Kahneman-Tversky Optimization)

不需要成對的偏好資料，只需要知道每個回應是好是壞。

### 6.3 ORPO (Odds Ratio Preference Optimization)

結合 SFT 和偏好優化到單一訓練階段。

In [ ]:
def ipo_loss(
    policy_chosen_logps: torch.Tensor,
    policy_rejected_logps: torch.Tensor,
    reference_chosen_logps: torch.Tensor,
    reference_rejected_logps: torch.Tensor,
    beta: float = 0.1,
) -> torch.Tensor:
    """
    IPO (Identity Preference Optimization) Loss
    
    使用 squared loss 而非 logistic loss，更加 robust
    """
    chosen_logratios = policy_chosen_logps - reference_chosen_logps
    rejected_logratios = policy_rejected_logps - reference_rejected_logps
    
    # IPO loss: (logratios_diff - 1/2β)^2
    logits = chosen_logratios - rejected_logratios
    loss = (logits - 1 / (2 * beta)) ** 2
    
    return loss.mean()


def kto_loss(
    policy_logps: torch.Tensor,
    reference_logps: torch.Tensor,
    is_desirable: torch.Tensor,  # 1 if good, 0 if bad
    kl_penalty: torch.Tensor,
    beta: float = 0.1,
) -> torch.Tensor:
    """
    KTO (Kahneman-Tversky Optimization) Loss
    
    不需要成對資料，只需要好/壞標籤
    基於前景理論 (Prospect Theory)
    """
    logratios = policy_logps - reference_logps
    
    # Desirable (good) responses: want to increase probability
    desirable_loss = 1 - F.sigmoid(beta * (logratios - kl_penalty))
    
    # Undesirable (bad) responses: want to decrease probability
    undesirable_loss = 1 - F.sigmoid(beta * (kl_penalty - logratios))
    
    # Combine based on labels
    loss = is_desirable * desirable_loss + (1 - is_desirable) * undesirable_loss
    
    return loss.mean()


print("DPO 變體比較：")
print("")
print("┌────────────────────────────────────────────────────────────────┐")
print("│  方法  │  資料需求           │  特點                          │")
print("├────────────────────────────────────────────────────────────────┤")
print("│  DPO   │  成對偏好 (A > B)  │  簡單有效，可能過擬合          │")
print("│  IPO   │  成對偏好 (A > B)  │  使用 squared loss，更穩定     │")
print("│  KTO   │  單一標籤 (好/壞)  │  不需成對，更易收集資料        │")
print("│  ORPO  │  成對偏好          │  結合 SFT，一階段訓練          │")
print("└────────────────────────────────────────────────────────────────┘")

## 7. 使用 TRL 庫進行實作

TRL (Transformer Reinforcement Learning) 是 Hugging Face 的官方 RLHF 庫。

### 7.1 TRL 核心組件

```python
# TRL 主要 Trainer 類別
from trl import (
    SFTTrainer,        # Supervised Fine-Tuning
    RewardTrainer,     # Reward Model Training
    PPOTrainer,        # PPO for RLHF
    DPOTrainer,        # Direct Preference Optimization
    KTOTrainer,        # Kahneman-Tversky Optimization
    ORPOTrainer,       # Odds Ratio Preference Optimization
)
```

In [ ]:
# TRL DPO 使用範例（概念展示）
trl_dpo_example = '''
# TRL DPO 完整範例
# 注意：需要安裝 trl: pip install trl

from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOConfig, DPOTrainer
from datasets import load_dataset

# 1. 載入模型和 tokenizer
model_name = "microsoft/DialoGPT-small"  # 或其他小型模型
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Reference model (frozen copy)
ref_model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. 準備偏好資料集
# 格式: {"prompt": str, "chosen": str, "rejected": str}
dataset = load_dataset("Anthropic/hh-rlhf", split="train[:1000]")

def format_dataset(example):
    # 根據資料集格式調整
    return {
        "prompt": example["prompt"],
        "chosen": example["chosen"],
        "rejected": example["rejected"],
    }

formatted_dataset = dataset.map(format_dataset)

# 3. 設定 DPO 訓練參數
training_args = DPOConfig(
    output_dir="./dpo_output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-7,
    beta=0.1,  # DPO temperature
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    fp16=True,  # 混合精度
    max_length=512,
    max_prompt_length=256,
)

# 4. 建立 DPOTrainer
trainer = DPOTrainer(
    model=model,
    ref_model=ref_model,
    args=training_args,
    train_dataset=formatted_dataset,
    tokenizer=tokenizer,
)

# 5. 開始訓練
trainer.train()

# 6. 儲存模型
trainer.save_model("./dpo_final_model")
'''

print("TRL DPO 使用範例：")
print("="*60)
print(trl_dpo_example)

In [ ]:
# TRL PPO 使用範例（概念展示）
trl_ppo_example = '''
# TRL PPO 完整範例
# 注意：PPO 比 DPO 複雜，需要更多設定

from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
import torch

# 1. 載入模型（需要 value head）
model_name = "gpt2"
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 2. 載入 Reward Model（需要預先訓練好）
reward_model = ...  # 您的 reward model

# 3. PPO 設定
ppo_config = PPOConfig(
    model_name=model_name,
    learning_rate=1e-5,
    batch_size=16,
    mini_batch_size=4,
    gradient_accumulation_steps=1,
    ppo_epochs=4,  # PPO update epochs per batch
    init_kl_coef=0.2,  # Initial KL penalty coefficient
    target_kl=6.0,  # Target KL divergence
)

# 4. 建立 PPOTrainer
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    ref_model=ref_model,
    tokenizer=tokenizer,
)

# 5. 訓練迴圈
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 128,
}

for epoch in range(num_epochs):
    for batch in dataloader:
        query_tensors = batch["input_ids"]
        
        # 生成回應
        response_tensors = ppo_trainer.generate(
            query_tensors,
            **generation_kwargs
        )
        
        # 計算 reward
        texts = [tokenizer.decode(r) for r in response_tensors]
        rewards = [reward_model(text) for text in texts]
        
        # PPO 更新
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        
        print(f"Epoch {epoch}, Mean reward: {torch.mean(rewards):.4f}")
'''

print("TRL PPO 使用範例：")
print("="*60)
print(trl_ppo_example)

## 8. 對齊評估方法

### 8.1 常見評估指標

```
┌─────────────────────────────────────────────────────────────────────┐
│                      對齊評估方法                                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  1. Win Rate（勝率）                                                │
│     - 讓人類或 GPT-4 比較兩個模型的輸出                              │
│     - 計算新模型「贏」的比例                                         │
│                                                                     │
│  2. Reward Model Score                                             │
│     - 使用 Reward Model 自動評分                                    │
│     - 注意：可能有 reward hacking                                   │
│                                                                     │
│  3. Benchmark 評估                                                  │
│     - MT-Bench：多輪對話能力                                        │
│     - AlpacaEval：指令遵循能力                                      │
│     - TruthfulQA：誠實性                                           │
│     - HarmBench：安全性                                            │
│                                                                     │
│  4. Human Evaluation                                               │
│     - 最可靠但最昂貴                                                │
│     - 評估維度：有幫助、無害、誠實                                   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

In [ ]:
def calculate_win_rate(model_a_scores: List[float], 
                       model_b_scores: List[float]) -> Dict[str, float]:
    """
    計算 Model A vs Model B 的勝率
    
    Args:
        model_a_scores: Model A 在每個問題上的分數
        model_b_scores: Model B 在每個問題上的分數
    
    Returns:
        win_rate, lose_rate, tie_rate
    """
    assert len(model_a_scores) == len(model_b_scores)
    n = len(model_a_scores)
    
    wins = sum(1 for a, b in zip(model_a_scores, model_b_scores) if a > b)
    losses = sum(1 for a, b in zip(model_a_scores, model_b_scores) if a < b)
    ties = sum(1 for a, b in zip(model_a_scores, model_b_scores) if a == b)
    
    return {
        'win_rate': wins / n,
        'lose_rate': losses / n,
        'tie_rate': ties / n,
        'wins': wins,
        'losses': losses,
        'ties': ties,
        'total': n,
    }


# 模擬評估結果
np.random.seed(42)

# 假設這是 RLHF 模型 vs 原始 SFT 模型的比較
n_samples = 100
rlhf_scores = np.random.normal(7.5, 1.5, n_samples)  # RLHF 模型較好
sft_scores = np.random.normal(6.0, 1.5, n_samples)   # SFT 模型

results = calculate_win_rate(rlhf_scores.tolist(), sft_scores.tolist())

print("RLHF Model vs SFT Model 評估結果：")
print(f"Win Rate:  {results['win_rate']:.1%} ({results['wins']}/{results['total']})")
print(f"Lose Rate: {results['lose_rate']:.1%} ({results['losses']}/{results['total']})")
print(f"Tie Rate:  {results['tie_rate']:.1%} ({results['ties']}/{results['total']})")

In [ ]:
# 視覺化對齊訓練過程
def visualize_alignment_training():
    """模擬並視覺化對齊訓練過程"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    steps = np.arange(0, 1000, 10)
    
    # 1. Reward 曲線
    reward_mean = 2 + 3 * (1 - np.exp(-steps / 300))
    reward_std = 0.5 * np.exp(-steps / 500) + 0.1
    
    axes[0, 0].plot(steps, reward_mean, 'b-', linewidth=2, label='Mean Reward')
    axes[0, 0].fill_between(steps, reward_mean - reward_std, reward_mean + reward_std, 
                            alpha=0.3, label='±1 std')
    axes[0, 0].set_xlabel('Training Steps')
    axes[0, 0].set_ylabel('Reward')
    axes[0, 0].set_title('Reward During Training')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. KL Divergence
    kl = 0.01 * steps ** 0.5
    kl_target = np.ones_like(steps) * 6.0
    
    axes[0, 1].plot(steps, kl, 'r-', linewidth=2, label='Actual KL')
    axes[0, 1].plot(steps, kl_target, 'k--', linewidth=1, label='Target KL')
    axes[0, 1].set_xlabel('Training Steps')
    axes[0, 1].set_ylabel('KL Divergence')
    axes[0, 1].set_title('KL Divergence from Reference Model')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Win Rate Over Time
    win_rate = 50 + 25 * (1 - np.exp(-steps / 400))
    
    axes[1, 0].plot(steps, win_rate, 'g-', linewidth=2)
    axes[1, 0].axhline(y=50, color='gray', linestyle='--', alpha=0.5, label='50% baseline')
    axes[1, 0].set_xlabel('Training Steps')
    axes[1, 0].set_ylabel('Win Rate (%)')
    axes[1, 0].set_title('Win Rate vs Base Model')
    axes[1, 0].set_ylim([40, 80])
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Reward vs KL Trade-off
    kl_values = np.linspace(0, 10, 100)
    reward_for_kl = 5 - 2 * np.exp(-kl_values / 2) + 0.3 * np.sqrt(kl_values)
    
    axes[1, 1].plot(kl_values, reward_for_kl, 'purple', linewidth=2)
    axes[1, 1].axvline(x=6, color='orange', linestyle='--', label='Typical KL target')
    axes[1, 1].set_xlabel('KL Divergence')
    axes[1, 1].set_ylabel('Expected Reward')
    axes[1, 1].set_title('Reward vs KL Trade-off (Pareto Frontier)')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # 標註最佳點
    optimal_kl = 6
    optimal_reward = 5 - 2 * np.exp(-optimal_kl / 2) + 0.3 * np.sqrt(optimal_kl)
    axes[1, 1].scatter([optimal_kl], [optimal_reward], color='red', s=100, zorder=5)
    axes[1, 1].annotate('Optimal', (optimal_kl, optimal_reward), 
                        textcoords="offset points", xytext=(10, 10))
    
    plt.tight_layout()
    plt.show()
    
    print("\n對齊訓練的關鍵觀察：")
    print("1. Reward 隨訓練增加，但有上限（避免 reward hacking）")
    print("2. KL divergence 需要控制，太大會導致能力退化")
    print("3. Win rate 是最終指標，反映實際對齊效果")
    print("4. Reward vs KL 有 trade-off，需要找到平衡點")

visualize_alignment_training()

## 9. 練習題

### 練習 1：實作完整的 Reward Model 訓練

使用提供的框架，實作一個可以訓練的 Reward Model。

In [ ]:
# 練習 1：實作 Reward Model 訓練
class PreferenceDataset(Dataset):
    """
    TODO: 實作偏好資料集
    
    每個樣本包含：
    - chosen_ids: 被偏好的回應的 token ids
    - rejected_ids: 被拒絕的回應的 token ids
    """
    def __init__(self, data: List[PreferenceExample], max_length: int = 128):
        self.data = data
        self.max_length = max_length
        # 簡化：使用字元級別的 tokenization
        self.char_to_idx = {chr(i): i for i in range(256)}
    
    def tokenize(self, text: str) -> torch.Tensor:
        """簡單的字元級 tokenization"""
        # TODO: 實作 tokenization
        # 1. 將文字轉換為 indices
        # 2. 截斷或填充到 max_length
        pass
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # TODO: 返回 chosen 和 rejected 的 token ids
        pass


def train_reward_model(model, dataset, num_epochs=5, lr=1e-4):
    """
    TODO: 實作 Reward Model 訓練迴圈
    
    步驟：
    1. 建立 DataLoader
    2. 建立 optimizer
    3. 訓練迴圈：
       - 前向傳播得到 chosen 和 rejected 的 reward
       - 計算 Bradley-Terry loss
       - 反向傳播並更新
       - 記錄 loss 和 accuracy
    """
    pass

print("練習 1：實作偏好資料集的 tokenize 和 __getitem__ 方法，")
print("然後實作 train_reward_model 函數")

### 練習 2：比較 DPO 和 IPO

在相同的偏好資料上，比較 DPO 和 IPO 的訓練穩定性。

In [ ]:
# 練習 2：比較 DPO 和 IPO
def compare_dpo_ipo():
    """
    TODO: 比較 DPO 和 IPO 的訓練行為
    
    步驟：
    1. 生成模擬的 log probability 資料
    2. 計算不同 beta 值下的 DPO loss 和 IPO loss
    3. 繪製 loss landscape
    4. 分析哪種方法更穩定
    """
    # 模擬資料
    n_samples = 100
    
    # 模擬 log probability differences
    # 正確情況：chosen 的 log ratio 高於 rejected
    log_ratio_diff = torch.randn(n_samples) + 0.5  # 平均正向
    
    # TODO: 計算不同 beta 值下的 loss
    betas = [0.01, 0.05, 0.1, 0.2, 0.5]
    
    # TODO: 繪製比較圖
    pass

print("練習 2：實作 compare_dpo_ipo 函數，")
print("比較兩種方法在不同 beta 值下的行為")

### 練習 3：實作簡單的 KTO

KTO 不需要成對資料，只需要好/壞標籤，實作並測試這種方法。

In [ ]:
# 練習 3：實作 KTO Trainer
@dataclass
class PointwiseExample:
    """單點資料（非成對）"""
    prompt: str
    response: str
    is_good: bool  # True = 好回應, False = 壞回應


class KTOTrainer:
    """
    TODO: 實作 KTO Trainer
    
    KTO 的關鍵：
    1. 不需要成對資料
    2. 基於 Kahneman-Tversky 前景理論
    3. 對好回應和壞回應使用不同的損失
    """
    def __init__(self, model, ref_model, beta=0.1, desirable_weight=1.0, undesirable_weight=1.0):
        self.model = model
        self.ref_model = ref_model
        self.beta = beta
        self.desirable_weight = desirable_weight
        self.undesirable_weight = undesirable_weight
    
    def compute_loss(self, logps, ref_logps, is_desirable):
        """
        TODO: 計算 KTO loss
        
        Hint:
        - log_ratio = logps - ref_logps
        - 對於好回應：loss = 1 - sigmoid(beta * (log_ratio - kl_penalty))
        - 對於壞回應：loss = 1 - sigmoid(beta * (kl_penalty - log_ratio))
        """
        pass

print("練習 3：實作 KTOTrainer 的 compute_loss 方法")

## 10. 總結

### 10.1 RLHF Pipeline 完整回顧

```
┌─────────────────────────────────────────────────────────────────────────┐
│                      RLHF 完整流程                                       │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ┌─────────────┐                                                        │
│  │ Pretrained  │                                                        │
│  │    LLM      │                                                        │
│  └──────┬──────┘                                                        │
│         │                                                               │
│         ▼                                                               │
│  ┌─────────────┐     ┌─────────────────────┐                            │
│  │    SFT      │◄────│ Demonstration Data  │                            │
│  │  Training   │     │ (Human-written)     │                            │
│  └──────┬──────┘     └─────────────────────┘                            │
│         │                                                               │
│         ▼                                                               │
│  ┌─────────────┐                                                        │
│  │  SFT Model  │────────────────────┐                                   │
│  │ (Reference) │                    │                                   │
│  └──────┬──────┘                    │                                   │
│         │                           │                                   │
│         │     Option A: RLHF        │     Option B: DPO                 │
│         │     ═══════════════       │     ═══════════════               │
│         ▼                           │                                   │
│  ┌─────────────┐                    │                                   │
│  │   Reward    │◄───Preference      │                                   │
│  │   Model     │    Data            │                                   │
│  └──────┬──────┘                    │                                   │
│         │                           │                                   │
│         ▼                           ▼                                   │
│  ┌─────────────┐             ┌─────────────┐                            │
│  │    PPO      │             │    DPO      │◄───Preference               │
│  │  Training   │             │  Training   │    Data                    │
│  └──────┬──────┘             └──────┬──────┘                            │
│         │                           │                                   │
│         └───────────┬───────────────┘                                   │
│                     │                                                   │
│                     ▼                                                   │
│              ┌─────────────┐                                            │
│              │   Aligned   │                                            │
│              │    Model    │                                            │
│              └─────────────┘                                            │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### 10.2 關鍵要點

| 概念 | 說明 |
|------|------|
| Alignment Gap | 預訓練目標與使用者期望的差距 |
| Reward Model | 學習人類偏好的評分模型 |
| PPO | 使用 clipped objective 的穩定 RL 算法 |
| KL Penalty | 防止模型偏離原始能力太遠 |
| DPO | 直接優化偏好，不需要 Reward Model |
| Win Rate | 評估對齊效果的核心指標 |

### 10.3 實際應用建議

1. **小規模實驗**：先用 DPO，簡單且穩定
2. **資料收集**：偏好資料品質比數量更重要
3. **評估**：多維度評估（helpfulness, harmlessness, honesty）
4. **迭代**：對齊是持續過程，需要不斷改進

In [ ]:
print("="*60)
print("RLHF 與 LLM 對齊 - 學習完成！")
print("="*60)
print("\n你已經學會：")
print("✓ 理解 LLM 對齊問題的本質")
print("✓ Reward Model 的訓練方法")
print("✓ PPO 在 LLM 上的應用")
print("✓ DPO 的原理與實作")
print("✓ 對齊效果的評估方法")
print("\n下一步學習建議：")
print("1. 使用 TRL 在真實資料上訓練 DPO")
print("2. 嘗試其他對齊方法（IPO, KTO, ORPO）")
print("3. 研究 Constitutional AI 和 RLAIF")
print("4. 探索多維度對齊（安全性、有幫助、誠實）")